# Марковская модель

## Библиотеки

In [1]:
import re

import numpy as np
import nltk
from nltk.tag import hmm
from nltk.corpus import brown
import pandas as pd

from collections import Counter

import nltk.lm as lm
from nltk.util import ngrams as nltk_ngrams
import numpy as np
import scipy.stats as st

from nltk.tokenize import RegexpTokenizer


## Brown Dataset HMM

### Dataset

In [2]:
nltk.download('brown')
english = re.compile('^[a-z]+$')

[nltk_data] Downloading package brown to /home/hp/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [3]:
tokens = []
for sent in brown.sents():
    for w in sent:
        w = w.lower()
        if english.match(w):
                tokens.append(w)
print(f'Number of tokens: {len(tokens)}')

Number of tokens: 981716


In [6]:
text = ' '.join(tokens)
len(text), text[:100]

(5579335,
 'the fulton county grand jury said friday an investigation of recent primary election produced no evi')

### Unsupervised обучение скрытой марковской модели (Алгоритм Баума-Велша)

In [7]:
vocab = sorted(list(set(text)))
len(vocab)

27

In [8]:
trainer = hmm.HiddenMarkovModelTrainer(range(2), vocab)

In [9]:
tagger = trainer.train_unsupervised([text[:5000]], max_iterations=50)

iteration 0 logprob -24142.453045979015
iteration 1 logprob -20551.003837963417
iteration 2 logprob -20549.04643655334
iteration 3 logprob -20546.535295938174
iteration 4 logprob -20543.311123785857
iteration 5 logprob -20539.173674525275
iteration 6 logprob -20533.875825370556
iteration 7 logprob -20527.121448297657
iteration 8 logprob -20518.570802262362
iteration 9 logprob -20507.857004676905
iteration 10 logprob -20494.614034394217
iteration 11 logprob -20478.510997862144
iteration 12 logprob -20459.284920294678
iteration 13 logprob -20436.772769448795
iteration 14 logprob -20410.95429513545
iteration 15 logprob -20381.997057466924
iteration 16 logprob -20350.235576335068
iteration 17 logprob -20316.01639170122
iteration 18 logprob -20279.50513136231
iteration 19 logprob -20240.704485852475
iteration 20 logprob -20199.808316221708
iteration 21 logprob -20157.745656919902
iteration 22 logprob -20116.36934632431
iteration 23 logprob -20077.685244997974
iteration 24 logprob -20042.924

#### Исследуем полученную модель


Матрица переходов $$\{a_{ij} = p(s_j|s_i)\}_{i,j = 1}^{|S|}$$

In [10]:
trans_matr = pd.DataFrame(data=np.array([
    [2 ** log_p for log_p in tagger._transitions[0]._data],
    [2 ** log_p for log_p in tagger._transitions[1]._data]
]),
                         columns=[0, 1],
                         index=[0, 1])
trans_matr

,0,1
0,0.267721,0.732279
1,0.749923,0.250077


Матрица выходных вероятностей $$\{ b_{ij} = p(x_j|s_i) \}_{i, j = 1}^{|S|, |X|}$$

In [11]:
out_matr = pd.DataFrame(data=np.array([
    [2 ** log_p for log_p in tagger._outputs[0]._data],
    [2 ** log_p for log_p in tagger._outputs[1]._data]
]),
                        index=[0, 1],
                        columns=vocab)
out_matr

,,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,0.294128,0.130406,6.883020e-09,0.000002,0.002901,2.138402e-01,1.879163e-08,0.001682,2.023943e-07,0.125689,...,3.696599e-34,6.107366e-09,5.277782e-07,0.053262,5.138490e-02,1.989185e-16,2.194478e-09,2.990145e-07,0.003044,1.348759e-34
1,0.031933,0.000033,1.943261e-02,0.057891,0.071116,8.112308e-13,4.898637e-02,0.024997,7.975449e-02,0.000007,...,8.096924e-04,1.121424e-01,9.108986e-02,0.121150,3.687532e-11,1.255023e-02,2.226654e-02,3.643310e-03,0.026031,1.619385e-03


### Supervised обучение скрытой марковской модели (максимум правдоподобия)

In [12]:
def make_tag(c):
    if c in 'aeiou':
        return (c,'1')
    else:
        return (c,'0')
supervised = [make_tag(c) for c in text]

In [13]:
tagger = trainer.train_supervised([supervised[:500]])

#### Исследуем полученную модель

Частоты совстречаемостей тегов

In [14]:
for t in tagger._transitions:
    print(t, tagger._transitions[t].__dict__)

0 {'_freqdist': FreqDist({'0': 199, '1': 142})}
1 {'_freqdist': FreqDist({'0': 142, '1': 16})}


Матрица переходов $$\{a_{ij} = p(s_j|s_i)\}_{i,j = 1}^{|S|}$$

In [15]:
trans_matr = pd.DataFrame(data=np.array([
    [tagger._transitions['0'].prob('0'), tagger._transitions['0'].prob('1')],
    [tagger._transitions['1'].prob('0'), tagger._transitions['1'].prob('1')]
]),
                         columns=[0, 1],
                         index=[0, 1])
trans_matr

,0,1
0,0.583578,0.416422
1,0.898734,0.101266


Матрица выходных вероятностей $$\{ b_{ij} = p(x_j|s_i) \}_{i, j = 1}^{|S|, |X|}$$

In [16]:
out_matr = pd.DataFrame(data=np.array([
    [tagger._outputs['0'].prob(c) for c in vocab],
    [tagger._outputs['1'].prob(c) for c in vocab]
]),
                        index=[0, 1],
                        columns=vocab)
out_matr

,,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,0.236842,0.000000,0.008772,0.05848,0.049708,0.000000,0.02924,0.023392,0.070175,0.000000,...,0.0,0.096491,0.052632,0.128655,0.000000,0.01462,0.017544,0.002924,0.035088,0.0
1,0.000000,0.177215,0.000000,0.00000,0.000000,0.348101,0.00000,0.000000,0.000000,0.208861,...,0.0,0.000000,0.000000,0.000000,0.101266,0.00000,0.000000,0.000000,0.000000,0.0


## Языковая модель

Построим языковую модель сначала вручную на простом синтетическом корпусе, затем обучим модель из пакета `nltk` на стихотворении "Дом, который построил Джек"

### Первый пример

In [17]:
text = 'SOS SOS ' + 'А Б ' * 100 + 'EOS'
tokens = text.split()
n = len(tokens)
tokens[:10]

['SOS', 'SOS', 'А', 'Б', 'А', 'Б', 'А', 'Б', 'А', 'Б']

In [18]:
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

In [19]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tokens, 3)

In [20]:
ngram_counts

{1: Counter({('SOS',): 2, ('А',): 100, ('Б',): 100, ('EOS',): 1}),
 2: Counter({('SOS', 'SOS'): 1,
          ('SOS', 'А'): 1,
          ('А', 'Б'): 100,
          ('Б', 'А'): 99,
          ('Б', 'EOS'): 1}),
 3: Counter({('SOS', 'SOS', 'А'): 1,
          ('SOS', 'А', 'Б'): 1,
          ('А', 'Б', 'А'): 99,
          ('Б', 'А', 'Б'): 99,
          ('А', 'Б', 'EOS'): 1})}

In [21]:
prefix_counts

{1: Counter({('*',): 203}),
 2: Counter({('SOS', '*'): 2, ('А', '*'): 100, ('Б', '*'): 100}),
 3: Counter({('SOS', 'SOS', '*'): 1,
          ('SOS', 'А', '*'): 1,
          ('А', 'Б', '*'): 100,
          ('Б', 'А', '*'): 99})}

#### N-граммы и их частотные вероятности

$$\hat p_i = \hat p(w_i)$$

In [22]:
def unigram_probas(ngram_counts):
    p1 = {}
    n = sum(ngram_counts[1].values())
    for w in ngram_counts[1]:
        p1[w] = ngram_counts[1][w] / n
    return p1

In [23]:
p1 = unigram_probas(ngram_counts)
p1

{('SOS',): 0.009852216748768473,
 ('А',): 0.49261083743842365,
 ('Б',): 0.49261083743842365,
 ('EOS',): 0.0049261083743842365}

$$\hat p_{i, i - 1} = \hat p(w_i|w_{i - 1})$$

In [24]:
def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2

In [25]:
p2 = bigram_probas(ngram_counts, prefix_counts)
p2

{'SOS|SOS': 0.5, 'А|SOS': 0.5, 'Б|А': 1.0, 'А|Б': 0.99, 'EOS|Б': 0.01}

$$\hat p_{i, i - 1, i - 2} = \hat p(w_i|w_{i - 1}, w_{i - 2})$$

In [26]:
def trigram_probas(ngram_counts, prefix_counts):
    p3 = {}
    for w in ngram_counts[3]:
        pre_w = w[:2] + tuple(['*'])
        p3[u'{2}|{1},{0}'.format(*w)] = ngram_counts[3][w] / prefix_counts[3][pre_w]
    return p3

In [27]:
p3 =  trigram_probas(ngram_counts, prefix_counts)
p3

{'А|SOS,SOS': 1.0,
 'Б|А,SOS': 1.0,
 'А|Б,А': 0.99,
 'Б|А,Б': 1.0,
 'EOS|Б,А': 0.01}

#### Проверка гипотезы, что триграммную модель можно свести к биграммной против правосторонней альтернативы

Статистика:
$$-2 \log (\prod_{i, j, k = 1}^m (\hat p_{ij} / \hat p_{ijk})^{n_{ijk}}) = \sum_{i, j, k}^m -2 n_{ijk} \log \hat p_{ij} + 2 n_{ijk} \log \hat p_{ijk} = \sum_{i = 3}^N -2 \log \hat p_{i,i - 1} + 2 \log \hat p_{i, i - 1, i - 2},$$
$$n_{ijk} = |\{X_t: X_t = O_i, X_{t + 1} = O_j, X_{t + 2} = O_k\}|$$

In [28]:
def chi2_statistic(p2, p3, tokens):
    stat2 = []
    stat3 = []
    n = len(tokens)
    for i in range(n - 2):
        w = tokens[i : i + 3]
        ngram3 = '{2}|{1},{0}'.format(*w)
        ngram2 = '{1}|{0}'.format(*w)

        stat2.append(np.log(p2[ngram2]))
        stat3.append(np.log(p3[ngram3]))
    return - 2 * np.sum(stat2) + 2 * np.sum(stat3)

In [29]:
m = len(p3)
stat = chi2_statistic(p2, p3, tokens)

In [30]:
print(f'p-value = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(stat)}')

p-value = 1.0


Значит триграммная модель бесполезна.

### Второй пример

In [31]:
text = 'SOS SOS ' + 'А Б Б А Б А Б А Б Б А А ' * 100
tokens = text.split()
tokens[:10]

['SOS', 'SOS', 'А', 'Б', 'Б', 'А', 'Б', 'А', 'Б', 'А']

In [32]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tokens, 3)

In [33]:
ngram_counts

{1: Counter({('SOS',): 2, ('А',): 600, ('Б',): 600}),
 2: Counter({('SOS', 'SOS'): 1,
          ('SOS', 'А'): 1,
          ('А', 'Б'): 400,
          ('Б', 'Б'): 200,
          ('Б', 'А'): 400,
          ('А', 'А'): 199}),
 3: Counter({('SOS', 'SOS', 'А'): 1,
          ('SOS', 'А', 'Б'): 1,
          ('А', 'Б', 'Б'): 200,
          ('Б', 'Б', 'А'): 200,
          ('Б', 'А', 'Б'): 300,
          ('А', 'Б', 'А'): 200,
          ('Б', 'А', 'А'): 100,
          ('А', 'А', 'А'): 99,
          ('А', 'А', 'Б'): 99})}

In [34]:
prefix_counts

{1: Counter({('*',): 1202}),
 2: Counter({('SOS', '*'): 2, ('А', '*'): 599, ('Б', '*'): 600}),
 3: Counter({('SOS', 'SOS', '*'): 1,
          ('SOS', 'А', '*'): 1,
          ('А', 'Б', '*'): 400,
          ('Б', 'Б', '*'): 200,
          ('Б', 'А', '*'): 400,
          ('А', 'А', '*'): 198})}

In [35]:
p1 = unigram_probas(ngram_counts)
p1

{('SOS',): 0.0016638935108153079,
 ('А',): 0.49916805324459235,
 ('Б',): 0.49916805324459235}

In [36]:
p2 = bigram_probas(ngram_counts, prefix_counts)
p2

{'SOS|SOS': 0.5,
 'А|SOS': 0.5,
 'Б|А': 0.667779632721202,
 'Б|Б': 0.3333333333333333,
 'А|Б': 0.6666666666666666,
 'А|А': 0.332220367278798}

In [37]:
p3 =  trigram_probas(ngram_counts, prefix_counts)
p3

{'А|SOS,SOS': 1.0,
 'Б|А,SOS': 1.0,
 'Б|Б,А': 0.5,
 'А|Б,Б': 1.0,
 'Б|А,Б': 0.75,
 'А|Б,А': 0.5,
 'А|А,Б': 0.25,
 'А|А,А': 0.5,
 'Б|А,А': 0.5}

#### Проверка той же гипотезы

In [38]:
stat = chi2_statistic(p2, p3, tokens)

In [39]:
print(f'p-value = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(stat)}')

p-value = 0.0


#### Сглаживание Лапласа

In [40]:
n1 = list(nltk_ngrams(tokens, 1))
n2 = list(nltk_ngrams(tokens, 2))
n3 = list(nltk_ngrams(tokens, 3))
n3[:10]

[('SOS', 'SOS', 'А'),
 ('SOS', 'А', 'Б'),
 ('А', 'Б', 'Б'),
 ('Б', 'Б', 'А'),
 ('Б', 'А', 'Б'),
 ('А', 'Б', 'А'),
 ('Б', 'А', 'Б'),
 ('А', 'Б', 'А'),
 ('Б', 'А', 'Б'),
 ('А', 'Б', 'Б')]

In [41]:
laplace = lm.Laplace(order=3)
laplace.fit([n1] + [n2] + [n3], vocabulary_text=list(set(tokens)))
regular_lm = lm.MLE(order=3)
regular_lm.fit([n1] + [n2] + [n3], vocabulary_text=list(set(tokens)))

#### Перплексия (Меньше $\rightarrow$ лучше)

In [42]:
laplace.perplexity(n1), regular_lm.perplexity(n1)

(2.024429736885131, 2.0224364471218337)

In [43]:
foo = [('b'), ('a'), ('r')]
laplace.perplexity(foo), regular_lm.perplexity(foo)

(1206.000000000001, inf)

#### Сглаженная по Лапласу оценка вероятности

$$p_L(w_i) = \frac{c_i + 1}{\sum_{i = 1}^v c_i + v}$$
$$p_L(w_i|w_j) = \frac{c_{ij} + 1}{\sum_{j=1}^v (c_{ij} + 1)} = \frac{c_{ij} + 1}{c_i + v}$$

$$p_L('А'|'SOS')$$

In [44]:
laplace.score('А', context=['SOS']), regular_lm.score('А', context=['SOS'])

(0.3333333333333333, 0.5)

$$p_L('SOS')$$

In [45]:
laplace.score('SOS'), regular_lm.score('SOS')

(0.0024875621890547263, 0.0016638935108153079)

#### n-граммы не встречаючиеся в тексте:

In [46]:
laplace.score('C', context=['SOS']), laplace.score('ыаываа', context=['B']), laplace.score('B')

(0.16666666666666666, 0.25, 0.0008291873963515755)

In [47]:
regular_lm.score('C', context=['SOS']), regular_lm.score('ыаываа', context=['B']), regular_lm.score('B')

(0.0, 0, 0.0)

### Генерация текста

In [49]:
rt = RegexpTokenizer(u'\w+')

In [50]:
with open('data/jack.txt') as f:
    text = f.read().lower()

In [51]:
tokens = rt.tokenize(text)
len(tokens), len(set(tokens))

(247, 57)

In [52]:
n1 = list(nltk_ngrams(tokens, 1) )
n2 = list(nltk_ngrams(tokens, 2))
n3 = list(nltk_ngrams(tokens, 3))

In [53]:
laplace = lm.Laplace(order=3)
laplace.fit([n1] + [n2] + [n3], vocabulary_text=list(set(tokens)))

In [54]:
' '.join(laplace.generate(50, random_seed=42))

'птица синица которая часто ворует пшеницу которая в тёмном чулане хранится в доме который построил джек а это старушка седая и строгая которая доит корову безрогую лягнувшую старого пса без хвоста который за шиворот треплет кота который пугает и ловит синицу которая часто ворует пшеницу которая в тёмном чулане хранится'

In [55]:
' '.join(laplace.generate(50, text_seed='вот дом который построил джек'.split()))

'а это корова безрогая лягнувшая старого пса без хвоста который за шиворот треплет кота который пугает и ловит синицу которая часто ворует пшеницу которая в тёмном чулане хранится в доме который построил джек а это весёлая птица синица которая часто ворует пшеницу которая в тёмном чулане хранится в доме который'

In [56]:
' '.join(laplace.generate(50, text_seed='привет как дела'.split()))

'ворует пшеницу которая в тёмном чулане хранится в доме который построил джек вот пёс без хвоста который за шиворот треплет кота который пугает и ловит синицу которая часто ворует пшеницу которая в тёмном чулане хранится в доме который построил джек а это старушка седая и строгая которая доит корову безрогую'